# torch

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from my_code.mysionna.channel.torch_version.awgn import AWGN
from my_code.mysionna.channel.torch_version.utils import expand_to_rank
class ApplyOFDMChannel(nn.Module):
    def __init__(self, add_awgn=True, dtype=torch.complex64):
        super(ApplyOFDMChannel, self).__init__()
        self.add_awgn = add_awgn
        self.dtype = dtype
        if self.add_awgn:
            self.awgn = AWGN(dtype=self.dtype)

    def forward(self, inputs):
        if self.add_awgn:
            x, h_freq, no = inputs
        else:
            x, h_freq = inputs

        # Apply the channel response
        x = expand_to_rank(x, h_freq.dim(), axis=1)
        y = torch.sum(torch.sum(h_freq * x, dim=4), dim=3)

        # Add AWGN if requested
        if self.add_awgn:
            y = self.awgn((y, no))

        return y

2024-07-13 15:57:07.059190: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-13 15:57:07.059239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-13 15:57:07.067334: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-13 15:57:07.094247: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-13 15:57:08.384894: W tensorflow/compiler/tf2

In [37]:
import tensorflow as tf
import numpy as np
import torch
from sionna.utils import expand_to_rank
from sionna.constants import GLOBAL_SEED_NUMBER
from my_code.mysionna.channel.torch_version.awgn import AWGN
from my_code.mysionna.channel.torch_version.utils import expand_to_rank

def generate_tensorflow_data(batch_size, num_tx, num_tx_ant, num_ofdm_symbols, fft_size):

    # 使用 TensorFlow 生成模拟数据
    x_real = tf.random.normal(shape=(batch_size, num_tx, num_tx_ant, num_ofdm_symbols, fft_size))
    x_imag = tf.random.normal(shape=(batch_size, num_tx, num_tx_ant, num_ofdm_symbols, fft_size))
    x_np_real = x_real.numpy()
    x_np_imag = x_imag.numpy()
    x_np = np.array(x_np_real + 1j * x_np_imag, dtype=np.complex64)
    return torch.tensor(x_np)

def generate_tensorflow_data_hfreq(batch_size, num_tx, num_tx_ant, num_ofdm_symbols, fft_size):

    # 使用 TensorFlow 生成模拟数据
    h_freq_real = tf.random.normal(shape=(batch_size, 2, 1, num_tx, num_tx_ant, num_ofdm_symbols, fft_size))
    h_freq_imag = tf.random.normal(shape=(batch_size, 2, 1, num_tx, num_tx_ant, num_ofdm_symbols, fft_size))
    h_freq_np_real = h_freq_real.numpy()
    h_freq_np_imag = h_freq_imag.numpy()
    h_freq_np = np.array(h_freq_np_real + 1j * h_freq_np_imag, dtype=np.complex64)
    return torch.tensor(h_freq_np)

def test_apply_ofdm_channel():
    # 设置CPU随机种子
    tf.random.set_seed(GLOBAL_SEED_NUMBER)

    # 定义输入数据的形状
    batch_size = 2
    num_tx = 3
    num_tx_ant = 2
    num_ofdm_symbols = 4
    fft_size = 16

    # 使用 TensorFlow 生成模拟数据 x 和 h_freq 并转换为 PyTorch 张量
    x = generate_tensorflow_data(batch_size, num_tx, num_tx_ant, num_ofdm_symbols, fft_size)
    h_freq = generate_tensorflow_data_hfreq(batch_size, num_tx, num_tx_ant, num_ofdm_symbols, fft_size)

    # 噪声功率 no
    no = torch.tensor(0.1, dtype=torch.float32)  # 这里假设噪声功率是一个标量

    # 创建 ApplyOFDMChannel 实例
    apply_channel = ApplyOFDMChannel(add_awgn=True, dtype=torch.complex64)

    # 执行前向传播
    y = apply_channel((x, h_freq, no))

    # 打印输入和输出
    print("输入数据 x:")
    print(x)
    print("\n频率响应 h_freq:")
    print(h_freq)
    print("\n噪声功率 no:")
    print(no)
    print("\n输出数据 y:")
    print(y)

# 运行测试函数
test_apply_ofdm_channel()



输入数据 x:
tensor([[[[[ 1.6052e-01+6.4974e-01j, -1.6598e+00+3.2791e-01j,
            -1.2321e+00-7.5198e-01j,  5.9717e-01-2.1430e-01j,
             1.0610e+00+5.2599e-01j, -1.3278e+00+1.1993e+00j,
            -2.7911e-01-1.2921e+00j, -2.1419e-02+1.1502e-01j,
            -1.5022e+00-4.2696e-01j,  3.0665e-01+6.5613e-01j,
             5.3554e-01+6.1071e-02j, -1.3167e+00+7.8160e-01j,
             7.3356e-01+1.5706e+00j, -1.1566e+00+1.1413e+00j,
             1.6611e+00-5.4925e-01j,  9.7467e-01+6.4841e-01j],
           [-1.0358e-01+1.0534e+00j,  2.0586e+00+1.8393e+00j,
            -2.1601e+00+7.8777e-01j, -7.1011e-01+2.2517e+00j,
            -1.4161e+00-1.1881e+00j, -6.1227e-01-3.3408e-01j,
            -2.5455e-01+9.5722e-01j, -1.8278e-01-2.4323e+00j,
            -9.3512e-01+7.0166e-01j, -7.4409e-01+1.1475e+00j,
             8.5019e-01+2.1080e-01j, -8.0659e-01+2.9560e-01j,
            -9.8049e-01-1.3067e+00j, -6.4699e-01+6.2680e-01j,
            -1.0294e+00-1.6480e+00j, -8.1056e-01+4.9235e-01j]

# tensorflow

In [4]:
#
# SPDX-FileCopyrightText: Copyright (c) 2021-2023 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: Apache-2.0
#
"""
Layer for applying OFDM channel: single-tap channel response in the frequency
domain
"""

import tensorflow as tf

from sionna.utils import expand_to_rank
from sionna.channel.awgn import AWGN
from sionna.constants import GLOBAL_SEED_NUMBER

class ApplyOFDMChannel(tf.keras.layers.Layer):
    # pylint: disable=line-too-long
    r"""ApplyOFDMChannel(add_awgn=True, dtype=tf.complex64, **kwargs)

    Apply single-tap channel frequency responses to channel inputs.

    This class inherits from the Keras `Layer` class and can be used as layer
    in a Keras model.

    For each OFDM symbol :math:`s` and subcarrier :math:`n`, the single-tap channel
    is applied as follows:

    .. math::
        y_{s,n} = \widehat{h}_{s, n} x_{s,n} + w_{s,n}

    where :math:`y_{s,n}` is the channel output computed by this layer,
    :math:`\widehat{h}_{s, n}` the frequency channel response (``h_freq``),
    :math:`x_{s,n}` the channel input ``x``, and :math:`w_{s,n}` the additive noise.

    For multiple-input multiple-output (MIMO) links, the channel output is computed for each antenna
    of each receiver and by summing over all the antennas of all transmitters.

    Parameters
    ----------

    add_awgn : bool
        If set to `False`, no white Gaussian noise is added.
        Defaults to `True`.

    dtype : tf.DType
        Complex datatype to use for internal processing and output. Defaults to
        `tf.complex64`.

    Input
    -----

    (x, h_freq, no) or (x, h_freq):
        Tuple:

    x :  [batch size, num_tx, num_tx_ant, num_ofdm_symbols, fft_size], tf.complex
        Channel inputs

    h_freq : [batch size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_ofdm_symbols, fft_size], tf.complex
        Channel frequency responses

    no : Scalar or Tensor, tf.float
        Scalar or tensor whose shape can be broadcast to the shape of the
        channel outputs:
        [batch size, num_rx, num_rx_ant, num_ofdm_symbols, fft_size].
        Only required if ``add_awgn`` is set to `True`.
        The noise power ``no`` is per complex dimension. If ``no`` is a
        scalar, noise of the same variance will be added to the outputs.
        If ``no`` is a tensor, it must have a shape that can be broadcast to
        the shape of the channel outputs. This allows, e.g., adding noise of
        different variance to each example in a batch. If ``no`` has a lower
        rank than the channel outputs, then ``no`` will be broadcast to the
        shape of the channel outputs by adding dummy dimensions after the
        last axis.

    Output
    -------
    y : [batch size, num_rx, num_rx_ant, num_ofdm_symbols, fft_size], tf.complex
        Channel outputs
    """

    def __init__(self, add_awgn=True, dtype=tf.complex64, **kwargs):

        super().__init__(trainable=False, dtype=dtype, **kwargs)

        self._add_awgn = add_awgn

    def build(self, input_shape): #pylint: disable=unused-argument

        if self._add_awgn:
            self._awgn = AWGN(dtype=self.dtype)

    def call(self, inputs):

        if self._add_awgn:
            x, h_freq, no = inputs
        else:
            x, h_freq = inputs

        # Apply the channel response
        x = expand_to_rank(x, h_freq.shape.rank, axis=1)
        y = tf.reduce_sum(tf.reduce_sum(h_freq*x, axis=4), axis=3)

        # Add AWGN if requested
        if self._add_awgn:
            y = self._awgn((y, no))

        return y


In [16]:
import tensorflow as tf
import numpy as np
from sionna.utils import expand_to_rank
from sionna.channel.awgn import AWGN  # 假设这里的路径和模块是正确的
from sionna.constants import GLOBAL_SEED_NUMBER
from sionna.channel.apply_ofdm_channel import ApplyOFDMChannel  # 假设这里的路径和模块是正确的

def test_apply_ofdm_channel():

    tf.random.set_seed(GLOBAL_SEED_NUMBER)

    # 定义输入数据的形状
    batch_size = 2
    num_tx = 3
    num_tx_ant = 2
    num_ofdm_symbols = 4
    fft_size = 16

    # 使用 TensorFlow 生成模拟数据 x
    x_real = tf.random.normal([batch_size, num_tx, num_tx_ant, num_ofdm_symbols, fft_size], dtype=tf.float32)
    x_imag = tf.random.normal([batch_size, num_tx, num_tx_ant, num_ofdm_symbols, fft_size], dtype=tf.float32)
    x = tf.complex(x_real, x_imag)

    # 定义频率响应的形状
    num_rx = 2
    num_rx_ant = 1

    # 使用 TensorFlow 生成模拟数据 h_freq
    h_freq_real = tf.random.normal([batch_size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_ofdm_symbols, fft_size], dtype=tf.float32)
    h_freq_imag = tf.random.normal([batch_size, num_rx, num_rx_ant, num_tx, num_tx_ant, num_ofdm_symbols, fft_size], dtype=tf.float32)
    h_freq = tf.complex(h_freq_real, h_freq_imag)

    # 噪声功率 no
    no = tf.constant(0.1, dtype=tf.float32)  # 这里假设噪声功率是一个标量

    # 创建 ApplyOFDMChannel 实例
    apply_channel = ApplyOFDMChannel(add_awgn=True, dtype=tf.complex64)

    # 执行前向传播
    y = apply_channel((x, h_freq, no))

    # 打印输入和输出
    print("输入数据 x:")
    print(x.numpy())
    print("\n频率响应 h_freq:")
    print(h_freq.numpy())
    print("\n噪声功率 no:")
    print(no.numpy())
    print("\n输出数据 y:")
    print(y.numpy())

# 运行测试函数
test_apply_ofdm_channel()

输入数据 x:
[[[[[ 1.60522267e-01+6.49735868e-01j -1.65976894e+00+3.27914953e-01j
     -1.23213315e+00-7.51981437e-01j  5.97165823e-01-2.14300945e-01j
      1.06098843e+00+5.25993168e-01j -1.32775724e+00+1.19929039e+00j
     -2.79114425e-01-1.29207397e+00j -2.14187484e-02+1.15016304e-01j
     -1.50224900e+00-4.26958561e-01j  3.06648910e-01+6.56127691e-01j
      5.35535812e-01+6.10709749e-02j -1.31672978e+00+7.81598032e-01j
      7.33561516e-01+1.57064915e+00j -1.15661943e+00+1.14134490e+00j
      1.66111290e+00-5.49247324e-01j  9.74668980e-01+6.48409069e-01j]
    [-1.03581309e-01+1.05343604e+00j  2.05857182e+00+1.83925390e+00j
     -2.16005945e+00+7.87771702e-01j -7.10112453e-01+2.25165606e+00j
     -1.41613340e+00-1.18814659e+00j -6.12270057e-01-3.34080130e-01j
     -2.54552037e-01+9.57218766e-01j -1.82776228e-01-2.43225408e+00j
     -9.35122967e-01+7.01658070e-01j -7.44086742e-01+1.14746249e+00j
      8.50186408e-01+2.10799098e-01j -8.06585610e-01+2.95597762e-01j
     -9.80492413e-01-1.30